In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pyemma
import pyemma.thermo.util
import mdshare
import shortcuts_thermo as shortcuts

# Analysing Umbrella sampling simulations 

In [ ]:
with np.load(mdshare.fetch('pyemma-tutorial-us-data.npz', working_directory='data')) as fh:
    # load biased data
    order_parameter_trajs = [fh['us_traj_%03d.npy' % i] for i in range(100)]
    rest_positions = fh['umbrella_centers'].tolist()
    spring_constants = fh['force_constants'].tolist()
    # load unbiased data
    adw_md_trajs = [fh['md_traj_%03d.npy' % i] for i in range(5)]

We use 20 different harmonic bias potentials (umbrella potentials).
They are defined by their sping constants $k^{(i)}$ and their 20 different rest positions $x^{(i)}$. 

$$b^{(i)}(x) = \frac{k^{(i)}}{2} \left\Vert x - x^{(i)} \right\Vert^2.$$

With each bias potential we ran 5 simulations which results in a total number of 20\*5=100 trajectories.

For each simulation $i$ we saved:

  * The spring constant $k{(i)}$ that was active in the simulation (variable `spring_constants[i]`)
  * The rest position $x{(i)}$ of the bias potential that was ative in the simulation (variable `rest_positions[i]`)
  * The time series $x(t)$ for the order parameter (variable `order_parameter_trajs[i]`)

In [ ]:
print('len(spring_constants) =', len(spring_constants))
print('len(rest_positions) =', len(rest_positions))
print('len(order_parameter_trajs) =', len(order_parameter_trajs))
print('order_parameter_trajs[0].shape =', order_parameter_trajs[0].shape)

Before we start with the analysis, let's get an overview over the data. 
For this, we make a histogram of every order parameter time series.
We also mark the umbrella rest positions in the same plot.

In [ ]:
for order_parameter_traj in order_parameter_trajs:
    plt.hist(order_parameter_traj)
plt.plot(rest_positions, np.ones_like(rest_positions)+100, 'xk')
plt.ylabel('counts')
plt.xlabel('order parameter / a.u.')

## 1. Using the bare metal API

To run the estimation methods, we have to define three quantities for every simulations:
    
   * (A) The thermodynamics state trajectory (`ttraj`), that contain the index of the bias potential used in the simulation.
   * (B) The bias energy trajectory (`btraj`) that contain the energies of every frame evaluated with all the bias potentials.
   * (C) The discrete microstate trajectory (`dtraj`).

### (A) The thermodynamic state trajectories

For every trajectory, we compute the index of the active bias.
Until now we didn't assign indices to the different bias potentials. We only have the lists `spring_constants` and `rest_positions` that contain the parameters that were used in the simulations. But because the numbering of the simualtions is arbitray, so is the ordering of `spring_constants` and `rest_positions`.

So let's assign indices to the potentials. We start by removing duplicates from the list:

In [ ]:
unique_spring_constants = np.unique(spring_constants)
unique_rest_positions = np.unique(rest_positions)

K = len(unique_rest_positions)
print('number of ensembles:', K)
print('unique_spring_constants', unique_spring_constants)

The order of the parameters in the lists now defines the indices, that is `unique_rest_positions[i]` is a parameter of the `i`'th potential.

Now that the order of the potentials is defined, we can compute the thermodynamic state trajectories.

In [ ]:
ttrajs = []
for i, rest_position in enumerate(rest_positions):
    # get index of active rest position in the list of all possible rest positions
    ensemble_index = np.where(rest_position == unique_rest_positions)[0][0] 
    # define the ttraj 
    ttraj = np.array([ensemble_index]*len(order_parameter_trajs[i]))
    ttrajs.append(ttraj) 

### (B) The bias energy trajectories

For every simulation we generate a 2-D array `btraj[t, k]` with the elements $$b^{(i)}(x_t) = \frac{k^{(i)}}{2} \left\Vert x_t - x^{(i)} \right\Vert^2.$$

Such an expression can be writtien very succinctly with `numpy` operations:

In [ ]:
btrajs = []
for order_parameter_traj in order_parameter_trajs:
    btraj = np.zeros((len(order_parameter_traj), K))
    for k in range(K):
        btraj[:, k] = 0.5*unique_spring_constants*(order_parameter_traj[:, 0] - unique_rest_positions[k])**2
    btrajs.append(btraj)
    
# The array indexing in order_parameter_traj[:, 0] is used to get rid of an unused extra array dimension.

### (C) The discrete microstate trajectory

Here we just use one of the clustering methods from PyEmma.
Since in this example, the state space is one-dimensional, we can just cluster the order parameter. 
In general it is not enough to cluster the order parameters but other system coordiantes have to be included into the definition of microstates.

In [ ]:
clustering_obj = pyemma.coordinates.cluster_regspace(order_parameter_trajs, max_centers=500, dmin=0.2)
dtrajs = clustering_obj.dtrajs

Now we're ready to run the estiamtors. We start with `wham`.
As explained in the lecture, WHAM assumes that the bias energy is constant on every microstate. However, above we computed bias energy values for every frame (conformation). We therefore have to coarse-grain the bias energy values. This is achieved by the utility functions `get_averaged_bias_matrix`.

In [ ]:
wham = pyemma.thermo.wham(ttrajs=ttrajs, dtrajs=dtrajs, 
                          bias=pyemma.thermo.util.get_averaged_bias_matrix(btrajs, dtrajs))

To quickly check the result, we plot minus the logarithm of the equilibrium distribution that was estimated by WHAM. 

In [ ]:
plt.plot(clustering_obj.clustercenters, -np.log(wham.pi_full_state), 'o')
plt.xlabel('order parameter / a.u.')
plt.ylabel('free energy / kT');

## 2. Using the user friendly API

The umbrella sampling API not only supports one-dimensional umbrella sampling with the bias

$$b^{(i)}(\mathbf{x}) = \frac{k^{(i)}}{2} \left\Vert \mathbf{x} - \mathbf{x}^{(i)} \right\Vert^2.$$

but also the more general case, where the bias energy is given with a quadratic form with a possibly non-symmetric force matrix:

$$b^{(i)}(\mathbf{x}) = \frac{1}{2} \left\langle \mathbf{x} - \mathbf{x}^{(i)} \middle\vert \mathbf{k}^{(i)} \middle\vert \mathbf{x} - \mathbf{x}^{(i)} \right\rangle.$$


For these simulation types, the `pyemma.thermo` module provides the API function

```python
def estimate_umbrella_sampling(
    us_trajs, us_dtrajs, us_centers, us_force_constants,
    md_trajs=None, md_dtrajs=None, kT=None,
    maxiter=10000, maxerr=1.0E-15, save_convergence_info=0,
    estimator='wham', lag=1, dt_traj='1 step', init=None):
    ...

```

In [ ]:
wham = pyemma.thermo.estimate_umbrella_sampling(
    order_parameter_trajs, dtrajs, rest_positions, spring_constants,
    maxiter=100000, maxerr=1.0E-15, save_convergence_info=50, estimator='wham')

Note also that we used the optinal parameter `save_convergence_info=50`. This instructs wham to track the convergence of the log-lokelihood and the change in the estimated free energy matrix.

In [ ]:
pyemma.plots.plot_convergence_info(wham)

Let's check the results:

In [ ]:
us_estimator = wham
adw_us_x, adw_us_f = shortcuts.adw_match_reference_to_binning(order_parameter_trajs, clustering_obj.clustercenters)
fig, ax = plt.subplots(1, 2, figsize=(2 * 6, 0.75*6))
ax[0].plot(
    clustering_obj.clustercenters, us_estimator.f_full_state, 's', markersize=10, label=us_estimator.name)
ax[0].plot(adw_us_x, adw_us_f, '-*', linewidth=2, markersize=9, color='black', label='Reference')
ax[0].set_xlabel(r"configuration state", fontsize=20)
ax[0].set_ylabel(r"f / kT", fontsize=20)
ax[1].plot(unique_rest_positions, us_estimator.f_therm, 's', markersize=10, label=us_estimator.name)
ax[1].set_xlabel(r"umbrella_center", fontsize=20)
ax[1].set_ylabel(r"f_therm / kT", fontsize=20)
for _ax in ax:
    _ax.tick_params(labelsize=15)
    _ax.set_ylim([0, 12])
    _ax.legend(loc=4, fontsize=10, fancybox=True, framealpha=0.5)
fig.tight_layout()